# 05-4. Comparativa LLMs y métricas en wandb

In [ ]:
!pip install wandb
!pip install openai
!pip install cohere
!pip install google-cloud-aiplatform google-genai --upgrade

## Setup wandb

In [ ]:
import wandb

wandb.init(project="comparison-llm", name = "comparison")
print(wandb.util.generate_id())
print(wandb.run)
table = wandb.Table(columns=["model", "time", "temperature", "max_output_tokens", "top_p", "top_k", "prompt", "response"])

## Setup OpenAI, Cohere and GOogle

In [ ]:
# TODO: insert YOUR_OPEN_API_KEY
from openai import OpenAI
import getpass

openai_api_key = getpass.getpass()
openai_client = OpenAI(api_key=openai_api_key) 

In [ ]:
# TODO: insert YOUR_COHERE_API_KEY
import cohere

cohere_api_key = getpass.getpass()
co = cohere.ClientV2(cohere_api_key) 

In [ ]:
# TODO: insert YOUR_GOOGLE_API_KEY
from google import genai
from google.genai.types import GenerateContentConfig

google_api_key = getpass.getpass() 

google_client = genai.Client(api_key=google_api_key)

## Setup prompts

In [ ]:
config = dict(
    temperature = 1.0,
    max_output_tokens = 128,
    top_p = 0.8,
    top_k = 40,
)

MODEL_GEMMA        = "gemma-3-1b-it"
MODEL_GOOGLE       = "gemini-2.0-flash"
MODEL_COHERE       = "command-a-03-2025"
MODEL_OPENAI_TOP   = "gpt-4.1"
MODEL_OPENAI_BASIC = "gpt-4-0613"

## Set queries

In [ ]:
import time

def call_llms(config, llm_call, show_activity = True):
  
  # Gemma
  t0 = time.perf_counter()

  response = google_client.models.generate_content(
    model=MODEL_GEMMA,
    contents=llm_call,
    config=GenerateContentConfig(
            temperature=config["temperature"],
            top_p=config["top_p"],
            top_k=config["top_k"],
            candidate_count=1,  
    )
  )

  res_gemma = response.text

  table.add_data(
       MODEL_GEMMA, 
       time.perf_counter() - t0, 
       config["temperature"], 
       config["max_output_tokens"], 
       config["top_p"], 
       config["top_k"], 
       llm_call, 
       res_gemma)
  
  # Gemini 2.0 flash
  t0 = time.perf_counter()

  response = google_client.models.generate_content(
    model=MODEL_GOOGLE, 
    contents=llm_call,
    config=GenerateContentConfig(
        temperature=config["temperature"],
        top_p=config["top_p"],
        top_k=config["top_k"],
        candidate_count=1,  
  )
)
  res_google = response.text

  table.add_data(
       MODEL_GOOGLE, 
       time.perf_counter() - t0, 
       config["temperature"], 
       config["max_output_tokens"], 
       config["top_p"], 
       config["top_k"], 
       llm_call, 
       res_google)
  
  # gpt-4.1
  t0 = time.perf_counter()

  res_openai = openai_client.responses.create(
      model=MODEL_OPENAI_TOP,
      input=llm_call
  )

  table.add_data(
    MODEL_OPENAI_TOP, 
    time.perf_counter() - t0, 
    config["temperature"], 
    config["max_output_tokens"], 
    config["top_p"], 
    config["top_k"], 
    llm_call, 
    res_openai.output_text
  )
  
  # gpt-4-0613
  t0 = time.perf_counter()

  res_openai = openai_client.chat.completions.create(
    model=MODEL_OPENAI_BASIC,
    temperature=config["temperature"],
    top_p=config["top_p"],
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": llm_call},
    ]
  )
  
  table.add_data(
    MODEL_OPENAI_BASIC, 
    time.perf_counter() - t0, 
    config["temperature"], 
    config["max_output_tokens"], 
    config["top_p"], 
    config["top_k"], 
    llm_call, 
    res_openai.choices[0].message.content
  )
  
  # Cohere
  t0 = time.perf_counter() 

  res_cohere = co.chat(
      model=MODEL_COHERE, 
      messages=[{"role": "user", "content": llm_call}],
      max_tokens=config["max_output_tokens"], 
      temperature=config["temperature"],
      k=config["top_k"],
      p=config["top_p"]
  )
  
  table.add_data(
    MODEL_COHERE, 
    time.perf_counter() - t0, 
    config["temperature"], 
    config["max_output_tokens"], 
    config["top_p"], 
    config["top_k"], 
    llm_call, 
    res_cohere.message.content[0].text 
  )
    
  ## Only show response from Gemini, not from all LLMs
  if show_activity:
    BOLD = "\033[1m"
    UNFORMAT = "\033[0m\x1B[0m"
    print(f"{BOLD}The call to Gemini LLM:{UNFORMAT}\n{llm_call}\n")
    print(f"{BOLD}The response:{UNFORMAT}")
    print(res_google)
        

  return res_google 

In [ ]:
queries = [
    "El planeta Tierra es ",
    "Implementar una función en Python para calcular los números de Fibonacci.",
    "Escribir una función en Rust que realice una exponenciación binaria.",
    "¿Cómo reservo memoria en C?",
    # "¿Cuáles son las diferencias entre Javascript y Python?",
    # "¿Cómo encuentro índices inválidos en Postgres?",
    # "¿Cómo puedes implementar una caché LRU (Least Recently Used) en Python?",
    # "¿Qué enfoque usarías para detectar y prevenir condiciones de carrera en una aplicación multiproceso?",
    # "¿Puedes explicar cómo funciona un algoritmo de árbol de decisión en el aprendizaje automático?",
    # "¿Cómo diseñarías una base de datos simple de almacenamiento clave-valor desde cero?",
    # "¿Cómo manejas situaciones de interbloqueo en la programación concurrente?",
    # "¿Cuál es la lógica detrás del algoritmo de búsqueda A*, y dónde se utiliza?",
    # "¿Cómo puedes diseñar un sistema de autocompletado eficiente?",
    # "¿Qué enfoque tomarías para diseñar un sistema seguro de gestión de sesiones en una aplicación web?",
    # "¿Cómo manejarías las colisiones en una tabla hash?",
    # "¿Cómo puedes implementar un balanceador de carga para un sistema distribuido?",
    "Escribe un cuento sobre un historiador que viaja en el tiempo y que presencie los eventos más significativos de la historia de la humanidad.",
    "Describe un día en la vida de un agente secreto que también es padre/madre a tiempo completo."
]

## Call models

In [ ]:
import time
from tqdm.auto import tqdm

for q in tqdm(queries):
    t0 = time.perf_counter()
    res = call_llms(config, q, False)

## Write to wandb

In [ ]:
wandb.log({"llm-comparison-180525": table})
table = wandb.Table(columns=["model", "time", "temperature", "max_output_tokens", "top_p", "top_k", "prompt", "response"])
wandb.finish()